# CS3237 Lab 3 Statistical Methods


|Student Nunmber|Name                  |
|:--------------|:---------------------|
|   A0200521E   |     Aryan Sarswat    |
|   A0200533Y   | Anurag S. Chatterjee |

In this lab you will also be do some experiments to familiarize yourself with the linear regression, Naive Bayes and Support Vector Machine library in SciKit Learn. 

Please work together as a team of 2 to complete this lab. You will need to submit ONE copy of this notebook per team, but please fill in the names of both team members above. This lab is worth 40 marks (but it is still 5% CA like your other labs):

**DO NOT SUBMIT MORE THAN ONE COPY OF THIS LAB!**

Let's now begin using statistical techniques in SciKit Learn. 

## 1. SciKit Learn Hands-on

We will now run some experiments to familiarize you with the statistical learning tools in SciKit Learn.

### 1.1 Linear Regression

Let's begin by playing around with the linear regression we did for the Boston Housing Dataset during the lecture. 

#### 1.1.1 Finding Better Correlations

_Question 0: (4 marks)_

In the lecture we took a quick look at correlating housing prices and poverty levels.  Using the code cell below:

    1. Recreate the regression example from the lecture Jupyter Notebook. Set random_state to 0 for train_test_split.
    2. Add code to find the correlation between housing prices and the other independent variables in the dataset. Leave out CHAS (Charles River dummy variable) and MEDV.
    3. As before save 33% of the data for testing and set random_state to 0.
    4. Create a new simple (single independent variable) regression model with the independent variable with the absolute highest correlation. If poverty levels is the highest, then choose the next highest.
    5. Compute and print the MSE for training data and testing data, and answer the questions after the code block.


In [2]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import sklearn

Load Data

In [3]:
"""
    Enter your code for part 1.1.1 here, and answer the questions
    after this code cell.
"""

from sklearn.datasets import load_boston
boston = load_boston()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

In [4]:
# Create dataframe named bos
bos = pd.DataFrame(boston.data)
bos.columns = boston.feature_names
bos.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [5]:
# Add labels
bos["PRICE"] = boston.target

Find correlations

In [6]:
# Find the correlation of other variables with price
correlation_matrix = bos.corr()

# Find absolute values of correlations in ascending order
print(abs(correlation_matrix['PRICE']).sort_values()) 

CHAS       0.175260
DIS        0.249929
B          0.333461
ZN         0.360445
AGE        0.376955
RAD        0.381626
CRIM       0.388305
NOX        0.427321
TAX        0.468536
INDUS      0.483725
PTRATIO    0.507787
RM         0.695360
LSTAT      0.737663
PRICE      1.000000
Name: PRICE, dtype: float64


Train test split on RM as it has the highest correlation

In [7]:
# Linear Regression In 1 Independant Variable
X = bos['RM'].values.reshape(-1, 1)
Y = bos['PRICE'].values.reshape(-1, 1)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, random_state=0) #Save 33% of the data for testing and set random state to 0

Train Linear Model

In [9]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train, Y_train)

LinearRegression()

Check metrics

In [10]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error
Y_pred_train = lm.predict(X_train)
Y_pred_test = lm.predict(X_test)
train_mse = mean_squared_error(Y_train, Y_pred_train)
test_mse = mean_squared_error(Y_test, Y_pred_test)
print("MSE for training data: %3.4f. MSE for testing data: %3.4f." % (train_mse, test_mse))

MSE for training data: 43.1318. MSE for testing data: 44.6075.


Answer the following questions between the \*\* markdowns so that your answers appear in bold.

***
_Question 1: Which independent variable has the highest correlation? Did it have any effect on your training and test accuracy scores? Why or why not?_ (2 marks)

***

**Answer: RM has the second highest correlation value which is 0.695360. The higher the correlation coefficient the strong the linear relationship between the variable and the target. Thus choosing the highest correlation value leads to a higher training and test accuracy score overall. Correlation coefficient is proportional to the error, so the higher the correlation you take, means that error between datapoints is smaller, so your training and testing accuracy scores will be better.** 

#### 1.1.2 Creating Multivariate Linear Regressions ####

SciKit learn can create linear regression models with multiple independent variables, and in this section we are going to explore how to do this, and whether or not it makes a difference in our Boston Dataset.

One way to create a multivariate model is to:

    1. Rank the independent variables by correlation, then create a linear model using the independent variable with the highest correlation. Measure the training and testing accuracy.
    2. Add in the independent variable with the next highest correlation and create a new linear model.  Measure the training and testing accuracy.
    3. Stop when either accuracy score levels off or goes down.

Answer the following questions to help you along with creating your multivariate model:

***

_Question 2: Explain what the following code fragment does. You may refer to NumPy and SciKit Learn documentation_ (2 marks)

```
bos['PRICE'].values.reshape(-1, 1)
```

**Answer: The above code fragment helps to give a new shape to the boston housing prices array without changing its data type. So it takes the series named "PRICE" from the boston housing dataset and converts into a numpy array, from of dimensions (n,) to dimensions (n,1) so basically -1 is a placeholder which helps numpy to calculate the dimensions for us instead of us having to specify it.**

_Question 3: Consult the NumPy documentation: What does the 'concatenate' function do? In particular what does 'axis=1' do?_ (2 marks)

**Answer: The numpy.concatenate function, numpy.concatenate((a1, a2, ...), axis=0, out=None, dtype=None, casting="same_kind") helps to join a sequence of numpy arrays of the same shape along a specified axis and merge it together into a single, compiled array. The a1 and a2 are arrays which you want to concatenate and must have the same shape, except in the dimension corresponding to axis (the first, by default). The out parameter, if provided, the destination to place the result. The shape must be correct, matching that of what concatenate would have returned if no out argument were specified. The axis parameter is the line along which the arrays will be joined/concatenated. If axis is None, arrays are flattened before use. Default is 0, and if the axis=1, then the numpy arrays a1 and a2, which are of the same shape, will be concatenated horizontally, along the column axis instead of the rows.**

_Question 4: Given your answers to Questions 2 and 3, what does the following code do?_ (2 marks)

```
import numpy as np

... Other code here ...

X1 = bos['INDUS'].values.reshape(-1, 1)
X2 = bos['CRIM'].values.reshape(-1, 1)
X = np.concatenate((X1, X2), axis = 1)
```

**Answer: The above combines the X1 and X2 arrays into a single array such that the entries of X are (X1_i, X2_i) where i is the index of the respective X1 and X2 entries. The above code fragment helps to give a new shape to the boston housing prices values, with the series named "INDUS" (proportion of non-retail business acres per town) and the series named "CRIM" (per capita crime rate by town) array without changing its data type. So it will convert these 2 array dimensions, from (n,) to dimensions (n,1) so basically -1 is a placeholder which helps numpy to calculate the dimensions for us instead of us having to specify it. Once this is done, then, the last code fragment will help concatente both the numpy array values concatenated horizontally, along the column axis.**

***

Use the following code cell to follow the steps above to create models with one, two and three independent variables, printing the training and testing accuracy each time. Note that you have to run _train_test_split_ for each model. Set the _random_state_ parameter in _train_test_split_ to 0 each time. (4 marks)

In [11]:
"""
    Enter your code for part 1.1.2 here. 
"""

#Get the top 3 independant variables
correlation_matrix = bos.corr()
print(abs(correlation_matrix['PRICE']).sort_values()[-5:])

INDUS      0.483725
PTRATIO    0.507787
RM         0.695360
LSTAT      0.737663
PRICE      1.000000
Name: PRICE, dtype: float64


In [ ]:
#For one independant varible refer to the first part of this question

In [12]:
# Linear Regression In 2 Independant Variables
X1 = bos['PTRATIO'].values.reshape(-1, 1)
X2 = bos['RM'].values.reshape(-1, 1)
X = np.concatenate((X1, X2), axis = 1)
Y = bos['PRICE'].values.reshape(-1, 1)

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, random_state=0) #Save 33% of the data for testing and set random state to 0

lm = LinearRegression()
lm.fit(X_train,Y_train)

LinearRegression()

In [14]:
from sklearn import metrics 
from sklearn.metrics import mean_squared_error
Y_pred_train = lm.predict(X_train) 
Y_pred_test = lm.predict(X_test)   
train_mse = mean_squared_error(Y_train, Y_pred_train)
test_mse = mean_squared_error(Y_test, Y_pred_test)
print("MSE for training data: %3.4f. MSE for testing data: %3.4f." % (train_mse, test_mse))

MSE for training data: 34.5563. MSE for testing data: 42.3877.


In [15]:
# Linear Regression In 3 Independant Variables
X1 = bos['INDUS'].values.reshape(-1, 1)
X2 = bos['PTRATIO'].values.reshape(-1, 1)
X3 = bos['RM'].values.reshape(-1, 1)
X = np.concatenate((X1, X2, X3), axis = 1)
Y = bos['PRICE'].values.reshape(-1, 1)

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, random_state=0) #Save 33% of the data for testing and set random state to 0

lm = LinearRegression()
lm.fit(X_train,Y_train)

LinearRegression()

In [17]:
from sklearn import metrics 
from sklearn.metrics import mean_squared_error
Y_pred_train = lm.predict(X_train) 
Y_pred_test = lm.predict(X_test)   
train_mse = mean_squared_error(Y_train, Y_pred_train)
test_mse = mean_squared_error(Y_test, Y_pred_test)
print("MSE for training data: %3.4f. MSE for testing data: %3.4f." % (train_mse, test_mse))

MSE for training data: 32.1271. MSE for testing data: 41.0605.


### 1.2 Creating a Naive Bayes Classifier ###

We will now look at how to create a Naive Bayes Classifier, and later on a Support Vector Machine classifier. We will also explore the use of _GridSearchCV_ to optimize the choice of parameters for the SVC.

#### 1.2.1 The Irises Dataset ###

In this lab we will use the irises dataset to classify four categories of irises (a species of flowers). We will consider four factors:

    1. Sepal length in cm
    2. Sepal width in cm
    3. Petal length in cm
    4. Petal width in cm

The image below shows what these mean:

![iris.png](attachment:image.png)

The code cell below loads up the Iris dataset, prints it out, then scales it.

In [18]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import numpy as np

iris_data = load_iris()
print("Iris Data:")
print(iris_data.data)
scaler = StandardScaler()
scaler.fit(iris_data.data)
X = scaler.transform(iris_data.data)
Y = iris_data.target



Iris Data:
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 

Answer the following questions:

_Question 5: What does 'StandardScaler' do? What other types of scalers are available? What is the advantage of scaling your inputs?_ (2 marks)

**Answer: StandardScalar calculates the mean and the standard deviation of the dataset and then scales all the samples in the dataset such that it is centered around the mean and scaled to the standard deviation i.e. unit variance. The advantage of scaling inputs lies in the fact that each feature will be given equal importance and the dataset will not give priority to one feature because it has larger values. Variables that are measured at different scales do not contribute equally to the model fitting & model learned function and might end up creating a bias. Thus, to deal with this potential problem feature-wise standardized (μ=0, σ=1) is usually used prior to model fitting, hence, StandardScalar() is used.**

#### 1.2.2 Creating a Naive Bayes Classifier Model

Recall that there are three major types of Naive Bayes classifiers:

    1. Gaussian
    2. Multinomial
    3. Bernoulli
    
_Question 6: What type of model should we use here? Why?_ (2 marks)

**Answer: We should use Gaussian Naive Bayes classifiers as the sepal and petal lengths and width are continuous in nature and the other classifiers (Multinomial and Bernoulli) do not support continuous values - with Multinomial supporting features which resembles a count and bernoulli for binary values.**


Now complete the code in the code cell below, following these specifications: (4 marks)

    1. Set aside 20% of the data for testing.
    2. Use the appropriate type of Naive Bayes Classifier, adding in whatever import statements you require here.
    3. Print out the training and testing accuracies.
    

In [19]:
"""
    Enter your code for part 1.2.2 here. 
"""
# Split the dataset
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state=0) #Sets aside 20% of data for testing with random state as 0

In [20]:
from sklearn.naive_bayes import GaussianNB

#Import and fit classifer

clf = GaussianNB()

clf.fit(X_train, Y_train)

GaussianNB()

In [21]:
# print out results
from sklearn import metrics
from sklearn.metrics import accuracy_score
Y_pred_train = clf.predict(X_train)
Y_pred_test = clf.predict(X_test)
train_accuracy = accuracy_score(Y_train, Y_pred_train, normalize=True)
test_accuracy = accuracy_score(Y_test, Y_pred_test, normalize=True)
print("Accuracy for training data: %3.4f. Accuracy for testing data: %3.4f." % (train_accuracy, test_accuracy))

Accuracy for training data: 0.9500. Accuracy for testing data: 0.9667.


#### 1.2.3 Using Pipelines ####

In the Naive Bayes Jupyter Notebook included with your Lecture 4 slides, we used a _Pipeline_ object to simplify our code. Using that example as a guide, rewrite your code above to use _Pipeline_. Some things to note:

    1. The code will not be exactly the same (it will be much simpler). For example we are not using a CountVectorizer nor a TfidfTransformer. So just follow the principle. Remember to put your StandardScaler into the Pipeline.
    2. When doing 'fit' on your model, you should input the _original_ data, not the scaled one, since we are incorporating the StandardScaler as part of our Pipeline.

**Hint: Section 1.3.2 below shows you how to create a Pipeline for SVM**

Use the code cell below to enter your new version using Pipelines. Remember to print out your training and testing accuracies. (4 marks)


In [22]:
"""
    Enter your code for part 1.2.3 here. 
"""

from sklearn.pipeline import Pipeline

iris_clf = Pipeline([('scaler', StandardScaler()),
                    ('clf', GaussianNB()), ])

In [23]:
X_train, X_test, Y_train, Y_test = train_test_split(iris_data.data, Y, test_size = 0.20, random_state=0) #Split 20% of data into testing data with random state as 0

iris_clf.fit(X_train,Y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('clf', GaussianNB())])

In [24]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
iris_clf.fit(X_train, Y_train)
Y_pred_train = iris_clf.predict(X_train)
Y_pred_test = iris_clf.predict(X_test)
train_accuracy = accuracy_score(Y_train, Y_pred_train, normalize=True)
test_accuracy = accuracy_score(Y_test, Y_pred_test, normalize=True)
print("Accuracy for training data: %3.4f. Accuracy for testing data: %3.4f." % (train_accuracy, test_accuracy))

Accuracy for training data: 0.9500. Accuracy for testing data: 0.9667.


## 1.3 Creating a Support Vector Machine Classifier ###

We will now create an SVM to perform our classification. There are two major SVM classifiers provided with SciKit Learn:

    1. LinearSVC: An SVM that uses a linear decision boundary to classify.
    2. SVC: An SVM that offers a wider variety of classification boundaries: Radial Basis Function (so-called 'kernel'), sigmoid, polynomials, and of course a linear boundary.
    
#### 1.3.1 Creating a Linear SVM ####

Using your code from 1.2.3 as a guide, create a new Pipeline to train a LinearSVC with the following parameters:

    - max_iter: 100000
    - loss: hinge
    - penalty: l2      (Note: This is 'el-two', and not 'twelve')
    
Use the code cell below to implement your SVM, printing out your training and testing accuraces. Please consult the SciKit Learn documentation on what these parameters mean. (4 marks)


In [25]:
"""
    Enter your code for part 1.3.1 here. 
"""
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

svc_clf = Pipeline([('scaler', StandardScaler()),
                   ('svm', LinearSVC(loss = 'hinge', penalty = 'l2',
                    random_state = 0, max_iter = 100000))])

In [26]:

X_train, X_test, Y_train, Y_test = train_test_split(iris_data.data, Y, test_size = 0.20, random_state=0) #Split 20% of data into testing data with random state as 0

svc_clf.fit(X_train,Y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm',
                 LinearSVC(loss='hinge', max_iter=100000, random_state=0))])

In [27]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
Y_pred_train = svc_clf.predict(X_train)
Y_pred_test = svc_clf.predict(X_test)
train_accuracy = accuracy_score(Y_train, Y_pred_train, normalize=True)
test_accuracy = accuracy_score(Y_test, Y_pred_test, normalize=True)
print("Accuracy for training data: %3.4f. Accuracy for testing data: %3.4f." % (train_accuracy, test_accuracy))

Accuracy for training data: 0.9083. Accuracy for testing data: 0.8667.


***
_Question 7: Play around with the loss and penalty parameters. E.g. try an 'l1' penalty with hinge loss, or 'l1' penalty with squared hinge loss. Does 'l2' work with the squared hinge loss function? Record your training and testing accuracies below_ (2 marks)

**Answer: Training Accuracy:0.9083 Testing Accuracy: 0.8667.**

***

#### 1.3.2 Autotuning Hyperparameters ####

In Question 7 you have played around with some of the hyperparameters for LinearSVC and may have found that it gives you different accuracy results. Selecting the right hyperparameters is always a challenge, but thankfully SciKit Learn gives us a very useful tool called "GridSearchCV". In the example below we see how to tweak the 'C' parameter, which controls penalties applied to the SVM parametrs, to various values of between 1 and 10. GridSearchCV will then select the C value that gives us the best possible training accuracy:

```
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

params = {'C':[1,10]}

svm_pipe_2 = Pipeline([('scaler', StandardScaler()), 
                    ('svm', GridSearchCV(svm.LinearSVC(max_iter = 100000), params)), ])
svm_pipe_2.fit(X_train_1, Y_train_1)

Y_train_pred_1 = svm_pipe_2.predict(X_train_1)
Y_test_pred_1 = svm_pipe_2.predict(X_test_1)

print("SVM Train Accuracy: %3.2f" % np.mean(Y_train_pred_1 == Y_train_1))
print("SVM Test Accuracy: %3.2f" % np.mean(Y_test_pred_1 == Y_test_1))
```

Note that the code above will not run because it's missing several variables, including X_train_1, etc. Notice that GridSearchCV is created in the Pipeline and takes svm.LinearSVC as a parameter.

The "param" variable is a dictionary that specifies which parameters to tune (in this case just simply 'C'), and what values to use (here \[1, 10\] means to use between 1 and 10). You can also specify labels instead of numeric values. E.g.:

```
params = {'kernel':('linear', 'poly')}
```

GridSearchCV will try 'linear' and 'poly', specified in the tuple after 'kernel', when tuning the SVM.

Use the code cell below to create a Pipeline that uses SVC (instead of LinearSVC), and applies GridSearchCV to tune the following hyperparameters:

    - C: From 1 to 10 as before
    - kernel: 'linear', 'poly', 'rbf', 'sigmoid'
    - decision_function_shape: 'ovr', 'ovo'
    
***
_Question 8: Consult the SVC documentation and write down below what each hyperparameter means. Also what is a 'decision function shape', and what is the difference between 'ovr' and 'ovo' in our decision function shape?_ (2 marks)

**Answer: C is the regularization parameter it is used to prevent overfitting by penalising very large weights. Kernel specifies the transformation which should be applied on the X train, this allows us to model some non-linearities. And lastly decision funtion shape decides how classication is performed. One vs one finds every permuation of classes and calculates the probability it is either of the two classes. One versus rest (ovr) calculates the probabilty of one class we being predicted against the rest, and this is repeated for all classes**

***

Remember to print out the training and testing accuracies. (4 marks)

In [28]:
"""
    Enter your code for part 1.3.2 here. 
"""

from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

params = {'C':[1,10],
         'kernel':('linear', 'poly', 'rbf', 'sigmoid'),
          'decision_function_shape': ('ovr', 'ovo')
         }

svm_pipe_2 = Pipeline([('scaler', StandardScaler()), 
                    ('svm', GridSearchCV(svm.SVC(max_iter = 100000), params))])


X_train_1, X_test_1, Y_train_1, Y_test_1 = train_test_split(iris_data.data, Y, test_size = 0.20, random_state=0)

svm_pipe_2.fit(X_train_1, Y_train_1)



Y_train_pred_1 = svm_pipe_2.predict(X_train_1)
Y_test_pred_1 = svm_pipe_2.predict(X_test_1)

print("SVM Train Accuracy: %3.2f" % np.mean(Y_train_pred_1 == Y_train_1))
print("SVM Test Accuracy: %3.2f" % np.mean(Y_test_pred_1 == Y_test_1))


SVM Train Accuracy: 0.96
SVM Test Accuracy: 1.00


### 1.4 Summary ###


***
_Question 9: Summarize in the table given below all the training and testing accuracies you've had in the previous section.  Give your thoughts on the performance of the various classifiers, and on using GridSearchCV to search for the right hyperparameters._

| Method            | Training Accuracy | Testing Accuracy |
|:-----------------:|:-----------------:|:----------------:|
| Linear Regression |     43.1318       |      44.6075     |
| LR (2 var)        |        34.5563    |   42.3877        |
| LR (3 var)        |    32.1271               |    41.0605              |
| Naive Bayes       |      0.9500             |     0.9667             |
| LinearSVC         |        0.9083           |       0.8667           |
| SVC (GridSearch)  |       0.96            |     1.00             |

***


**FOR LAB TA ONLY**

Q0:        ___/ 4

Q1:        ___/ 2

Q2:        ___/ 2

Q3:        ___/ 2

Q4:        ___/ 2

Q4 Code:   ___/ 4 

Q5:        ___/ 2

Q6:        ___/ 2

Q6 Code:   ___/ 4

Q6 Code 2: ___/ 4 

Q7:        ___/ 2

Q8:        ___/ 2

Q8 Code:   ___/ 4

Q9:        ___/ 4

Total:     ___/ 40